# 🔥 TPS-JUN22 Unsupervised Clustering + GBDT
Hello Kaggle After a few days off from the competition.

I saw multiple **Notebooks** using **Classification** models to enhance the predictions provided by the Clustering Algorithm; It took me a while to understand the technique due to majority of the **Notebooks** and **Code** don't have good documentation, in this **Notebbook** I will explain to my best how this type of technique works.

**This will be the strategy will follow on this Notebook:**

* Loading all the information available, and quickly explore it.
* We identify the Clusters using a Clustering Algortihm (Bayesian Gaussian Mixture).
* We create a sumset of the dataset, based on the Clusters with the highest probability to be that cluster (Greather than 80%).
* We build a ML model passing the high probability records and the cluster number as the target.
* We predict the cluster based on the new trained model predictions (GBTD or Other).


**Credits and References:**
* https://www.kaggle.com/code/adaubas/tps-jul22-lgbm-extratree-qda-soft-voting/notebook?scriptVersionId=100798978
* https://www.kaggle.com/code/cv13j0/tps-jun22-unsupervised-clustering-with-keras
* ...

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jul-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-jul-2022/data.csv


In [2]:
%%time
# Model Libraries for K-Means
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer

# Scientific Libraries for Calculations
from scipy.spatial.distance import cdist

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Import Machine Learning Model
import lightgbm as lgb

CPU times: user 1.26 s, sys: 468 ms, total: 1.73 s
Wall time: 2.25 s


In [3]:
%%time
# I like to disable my Notebook Warnings To Reduce Noice.
import warnings
warnings.filterwarnings('ignore')

CPU times: user 34 µs, sys: 6 µs, total: 40 µs
Wall time: 44.3 µs


In [4]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 25
NCOLS = 20
# Main data location path...
BASE_PATH = '...'

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.39 µs


In [5]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

CPU times: user 72 µs, sys: 13 µs, total: 85 µs
Wall time: 89.2 µs


In [6]:
%%time
# Placeholder, Describe Code...
dataset = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2022/data.csv')
submission = pd.read_csv("../input/tabular-playground-series-jul-2022/sample_submission.csv")

CPU times: user 703 ms, sys: 86.4 ms, total: 790 ms
Wall time: 1.37 s


In [7]:
%%time
# Placeholder, Describe Code...
dataset.info(verbose = False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98000 entries, 0 to 97999
Columns: 30 entries, id to f_28
dtypes: float64(22), int64(8)
memory usage: 22.4 MB
CPU times: user 7.82 ms, sys: 1.12 ms, total: 8.94 ms
Wall time: 18.2 ms


In [8]:
%%time
# Placeholder, Describe Code...
dataset.head()

CPU times: user 113 µs, sys: 19 µs, total: 132 µs
Wall time: 137 µs


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28
0,0,-0.39,-0.91,0.65,0.59,-0.83,0.73,2.26,2,13,...,-0.48,-0.76,-0.76,-1.09,1.14,-0.88,1.14,1.31,1.46,0.81
1,1,-0.69,-0.45,0.65,1.00,-1.65,0.86,-0.09,2,3,...,-0.43,-0.09,-1.78,-0.84,0.46,1.76,-0.28,-0.85,0.56,-2.68
2,2,0.81,0.32,-1.17,-0.62,0.11,0.78,1.99,5,11,...,-0.41,-1.60,1.19,3.27,-0.09,-2.17,-0.97,1.34,-1.11,-3.63
3,3,-0.50,0.23,0.26,0.23,0.42,-1.22,0.14,6,2,...,0.62,1.29,0.53,1.04,-2.04,1.44,-1.90,-0.63,-0.05,0.24
4,4,-0.67,-1.04,-0.27,-1.83,-0.29,-1.85,0.78,8,7,...,-1.63,-0.43,0.32,0.28,-2.44,1.47,-1.04,1.60,-0.41,-1.99


In [9]:
%%time
# Placeholder, Describe Code...
dataset.describe()

CPU times: user 175 ms, sys: 1.27 ms, total: 177 ms
Wall time: 177 ms


,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_19,f_20,f_21,f_22,f_23,f_24,f_25,f_26,f_27,f_28
count,"98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00",...,"98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00","98,000.00"
mean,"48,999.50",0.00,0.01,-0.00,-0.00,-0.00,-0.00,-0.00,5.55,6.76,...,-0.00,-0.00,-0.00,-0.04,-0.22,0.17,-0.06,-0.06,0.10,-0.23
std,"28,290.31",1.00,1.00,1.00,1.00,1.00,1.00,1.00,3.69,4.15,...,1.00,1.00,1.00,1.48,1.49,1.54,1.58,1.43,1.31,1.53
min,0.00,-4.73,-4.20,-4.38,-4.01,-4.54,-4.30,-4.89,0.00,0.00,...,-4.89,-4.73,-4.44,-6.87,-8.23,-7.79,-6.59,-7.38,-7.34,-6.95
25%,"24,499.75",-0.68,-0.67,-0.67,-0.67,-0.68,-0.68,-0.68,3.00,4.00,...,-0.68,-0.68,-0.68,-1.02,-1.20,-0.90,-1.13,-0.98,-0.75,-1.26
50%,"48,999.50",0.00,0.01,-0.00,-0.00,-0.00,0.00,-0.00,5.00,6.00,...,-0.00,-0.00,0.00,-0.06,-0.22,0.17,-0.10,-0.07,0.08,-0.27
75%,"73,499.25",0.68,0.68,0.68,0.67,0.68,0.67,0.67,8.00,9.00,...,0.67,0.68,0.68,0.93,0.76,1.22,0.99,0.84,0.93,0.77
max,"97,999.00",4.49,4.32,4.56,4.40,4.05,4.71,4.00,32.00,30.00,...,4.56,4.40,4.14,6.52,6.05,7.53,7.54,7.01,7.21,6.98


In [10]:
%%time
# Placeholder, Describe Code...
dataset.nunique()

CPU times: user 105 ms, sys: 0 ns, total: 105 ms
Wall time: 105 ms


id      98000
f_00    98000
f_01    98000
f_02    98000
f_03    98000
        ...  
f_24    98000
f_25    98000
f_26    98000
f_27    98000
f_28    98000
Length: 30, dtype: int64

In [11]:
%%time
# Placeholder, Describe Code...
dataset.isnull().sum().sum()

CPU times: user 7.58 ms, sys: 0 ns, total: 7.58 ms
Wall time: 6.78 ms


0

In [12]:
%%time
# Placeholder, Describe Code...
dataset.isnull().sum()

CPU times: user 7.51 ms, sys: 83 µs, total: 7.59 ms
Wall time: 6.68 ms


id      0
f_00    0
f_01    0
f_02    0
f_03    0
       ..
f_24    0
f_25    0
f_26    0
f_27    0
f_28    0
Length: 30, dtype: int64

In [13]:
%%time
# Placeholder, Describe Code...
categorical_cols = ['f_07','f_08','f_09','f_10','f_11','f_12','f_13']

CPU times: user 0 ns, sys: 4 µs, total: 4 µs
Wall time: 8.11 µs


In [14]:
%%time
# Create a Feature List...
ignore = ['id']
feat = [feat for feat in dataset.columns if feat not in ignore]

# Overwrite the feature by this default list...
feat =['f_07','f_08', 'f_09', 'f_10','f_11', 'f_12', 'f_13', 'f_22','f_23', 'f_24', 'f_25','f_26','f_27', 'f_28']

CPU times: user 33 µs, sys: 5 µs, total: 38 µs
Wall time: 42 µs


In [15]:
%%time
# Placeholder, Describe Code...
X = dataset.copy(deep = True)
X[feat] = StandardScaler().fit(X[feat]).transform(X[feat])
X[feat] = PowerTransformer().fit(X[feat]).transform(X[feat])

CPU times: user 2.11 s, sys: 12.3 ms, total: 2.12 s
Wall time: 2.12 s


In [16]:
%%time
# Set a baseline with K-Means
# 10 clusters
n_components = 7
# Runs in parallel All CPUs
# Train Gaussian Mixture.

bgm = BayesianGaussianMixture(n_components = n_components, 
                              covariance_type = 'full', 
                              tol = 0.001, 
                              reg_covar = 1e-06, 
                              max_iter = 100, 
                              n_init = 1, 
                              init_params = 'kmeans', 
                              weight_concentration_prior_type = 'dirichlet_process', 
                              weight_concentration_prior = None, 
                              mean_precision_prior = None, 
                              mean_prior = None, 
                              degrees_of_freedom_prior = None, 
                              covariance_prior = None, 
                              random_state = 1, 
                              warm_start = False, 
                              verbose = 0, 
                              verbose_interval = 10)

bgm.fit(X[feat])
bgm_predictions = bgm.predict(X[feat])
bgm_predictions_proba = bgm.predict_proba(X[feat])

CPU times: user 1min 8s, sys: 50.1 s, total: 1min 58s
Wall time: 30.3 s


In [17]:
%%time
# Placeholder, Describe Code...
submission["Predicted"] = bgm_predictions
submission.to_csv("submission.csv", index = False)

CPU times: user 156 ms, sys: 4.81 ms, total: 161 ms
Wall time: 162 ms


In [18]:
%%time
# Placeholder, Describe Code...
submission.head(10)

CPU times: user 77 µs, sys: 50 µs, total: 127 µs
Wall time: 131 µs


,Id,Predicted
0,0,6
1,1,5
2,2,6
3,3,3
4,4,1
5,5,5
6,6,3
7,7,6
8,8,5
9,9,6


In [19]:
%%time

# Building a high confidence dataset for training...
X['predictions'] = bgm_predictions
X['predict_proba'] = 0

CLUSTERS = 7 # Number of Clusters or Components used...

for n in range(CLUSTERS):
    # Loop over all the clusters, and creates a probability column for each cluster or component...
    X[f'predict_proba_{n}'] = bgm_predictions_proba[:,n] # Write the probability for each cluster as a new feature
    X.loc[X.predictions == n,'predict_proba'] = X[f'predict_proba_{n}'] # Extract the probility of the estimated cluster.

CPU times: user 24.8 ms, sys: 460 µs, total: 25.3 ms
Wall time: 24.1 ms


In [20]:
%%time

pct_data = 0
min_probability = 0.8 # Set the minimun probability allowed to create the new training dataset...

# Generate a list of indexes with higher confidence or probabilities

high_confidence_idx = np.array([])
for cluster in range(CLUSTERS):
    median_probability = X[X['predictions'] == cluster]['predict_proba'].median()
    idx = X[(X['predictions'] == cluster) & (X['predict_proba'] > min_probability)].index
    pct_data = len(idx) / len(X[(X['predictions'] == cluster)])
    print(f'Cluster: {cluster}, Median Probability: {median_probability : .3f}, Data Pct: {pct_data: .2f}')                           
    high_confidence_idx = np.concatenate((high_confidence_idx, idx))
                     

Cluster: 0, Median Probability:  0.919, Data Pct:  0.67
Cluster: 1, Median Probability:  0.885, Data Pct:  0.62
Cluster: 2, Median Probability:  0.924, Data Pct:  0.69
Cluster: 3, Median Probability:  0.982, Data Pct:  0.82
Cluster: 4, Median Probability:  0.859, Data Pct:  0.59
Cluster: 5, Median Probability:  0.754, Data Pct:  0.43
Cluster: 6, Median Probability:  0.944, Data Pct:  0.71
CPU times: user 96.9 ms, sys: 6.35 ms, total: 103 ms
Wall time: 102 ms


# LGBM Model ...

In [21]:
%%time
N_FOLDS = 10

X_trn = X.loc[high_confidence_idx][feat]
label = X.loc[high_confidence_idx]['predictions']


params_lgb = {'learning_rate': 0.07,
              'objective': 'multiclass',
              'boosting': 'gbdt',
              'verbosity': -1,
              'n_jobs': -1,
              'num_classes':CLUSTERS} 

model_list = []
gkf = StratifiedKFold(N_FOLDS)
lgbm_predictions_prob = 0

for fold, (train_idx, valid_idx) in enumerate(gkf.split(X_trn,label)):  
    print(f'FOLD:{fold}...')
    trn_dataset = lgb.Dataset(X_trn.iloc[train_idx],label.iloc[train_idx],feature_name = feat)
    val_dataset = lgb.Dataset(X_trn.iloc[valid_idx],label.iloc[valid_idx],feature_name = feat)
    
    model = lgb.train(params = params_lgb, 
                      train_set = trn_dataset, 
                      valid_sets = val_dataset, 
                      num_boost_round = 5000, 
                      callbacks = [lgb.early_stopping(stopping_rounds = 300, verbose = True), lgb.log_evaluation(period = 200)])  
    
    model_list.append(model)
    
    y_pred_proba = model.predict(X_trn.iloc[valid_idx])
    y_pred = np.argmax(y_pred_proba, axis = 1)
    
    score = balanced_accuracy_score(label.iloc[valid_idx], y_pred)
    auc = roc_auc_score(label.iloc[valid_idx], y_pred_proba, average = "weighted", multi_class = "ovo")
    
    lgbm_predictions_prob += model.predict(X[feat]) / N_FOLDS
    
    print(f'LGBM AUC : {score:.3f} | ACC : {auc:.1%}\n')
    print('.' * 10)
    print('')

FOLD:0...
Training until validation scores don't improve for 300 rounds
[200]	valid_0's multi_logloss: 0.0207654
[400]	valid_0's multi_logloss: 0.0136906
[600]	valid_0's multi_logloss: 0.0130861
[800]	valid_0's multi_logloss: 0.0137368
Early stopping, best iteration is:
[563]	valid_0's multi_logloss: 0.0128833
LGBM AUC : 0.995 | ACC : 100.0%

..........

FOLD:1...
Training until validation scores don't improve for 300 rounds
[200]	valid_0's multi_logloss: 0.0180453
[400]	valid_0's multi_logloss: 0.0100428
[600]	valid_0's multi_logloss: 0.00954692
[800]	valid_0's multi_logloss: 0.0103506
Early stopping, best iteration is:
[504]	valid_0's multi_logloss: 0.00914013
LGBM AUC : 0.996 | ACC : 100.0%

..........

FOLD:2...
Training until validation scores don't improve for 300 rounds
[200]	valid_0's multi_logloss: 0.0223837
[400]	valid_0's multi_logloss: 0.0139131
[600]	valid_0's multi_logloss: 0.0133291
Early stopping, best iteration is:
[494]	valid_0's multi_logloss: 0.0131472
LGBM AUC : 0.

# Extra Trees ...

In [22]:
%%time
N_FOLDS = 10
SEED = 15

X_trn = X.loc[high_confidence_idx][feat]
label = X.loc[high_confidence_idx]['predictions']

model_list = []
gkf = StratifiedKFold(N_FOLDS)
extra_trees_predictions_prob = 0

for fold, (train_idx, valid_idx) in enumerate(gkf.split(X_trn,label)):  
    print(f'FOLD:{fold}...')
    X_train, y_train = X_trn.iloc[train_idx], label.iloc[train_idx]
    X_valid, y_valid = X_trn.iloc[valid_idx], label.iloc[valid_idx]
    
    model = ExtraTreesClassifier(n_estimators = 100, random_state = SEED)
    model.fit(X_train, y_train)
    model_list.append(model)
    
    y_pred = model.predict(X_valid)
    y_pred_proba = model.predict_proba(X_valid)
    
    score = balanced_accuracy_score(y_valid, y_pred)
    auc = roc_auc_score(y_valid, y_pred_proba, average = "weighted", multi_class = "ovo")
    
    extra_trees_predictions_prob += model.predict_proba(X[feat]) / N_FOLDS
    
    print(f'Extra Trees AUC : {score:.3f} | ACC : {auc:.1%}')
    print('.' * 10)
    print('')

FOLD:0...
Extra Trees AUC : 0.975 | ACC : 99.9%
..........

FOLD:1...
Extra Trees AUC : 0.980 | ACC : 100.0%
..........

FOLD:2...
Extra Trees AUC : 0.974 | ACC : 99.9%
..........

FOLD:3...
Extra Trees AUC : 0.976 | ACC : 99.9%
..........

FOLD:4...
Extra Trees AUC : 0.974 | ACC : 99.9%
..........

FOLD:5...
Extra Trees AUC : 0.978 | ACC : 99.9%
..........

FOLD:6...
Extra Trees AUC : 0.974 | ACC : 99.9%
..........

FOLD:7...
Extra Trees AUC : 0.973 | ACC : 99.9%
..........

FOLD:8...
Extra Trees AUC : 0.973 | ACC : 99.9%
..........

FOLD:9...
Extra Trees AUC : 0.974 | ACC : 100.0%
..........

CPU times: user 1min 31s, sys: 2.15 s, total: 1min 33s
Wall time: 1min 33s


In [23]:
%%time
def soft_voting_predictions(prediction_prob, df):
    '''
    
    '''
    values = list(range(CLUSTERS))
    pred_test = pd.DataFrame(np.zeros((dataset.shape[0], 7)), columns = values)
    
    for model, probability in enumerate(prediction_prob):
        max_value = np.argmax(probability, axis = 1)
        df[f'pred_model_{model}'] = max_value
        
        # Sort the Predictions...
        pred_keys = df[f'pred_model_{model}'].value_counts().index.to_list()
        pred_dict = dict(zip(pred_keys, values))
        df[f'pred_model_{model}'] = df[f'pred_model_{model}'].map(pred_dict)
        
        pred_new = pd.DataFrame(probability).rename(columns = pred_dict)   
        pred_new = pred_new.reindex(sorted(pred_new.columns), axis=1)
        
        # Soft Voting by Probabiliy Addition
        pred_test += pred_new 
                                      
    return np.argmax(np.array(pred_test), axis = 1)                                 

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


In [24]:
%%time
soft_vote_results = soft_voting_predictions([lgbm_predictions_prob, extra_trees_predictions_prob], dataset)

CPU times: user 32.7 ms, sys: 991 µs, total: 33.6 ms
Wall time: 33.7 ms


In [25]:
%%time
soft_vote_results

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 9.3 µs


array([3, 6, 3, ..., 1, 5, 2])

In [26]:
%%time
submission_lgbm = pd.read_csv("../input/tabular-playground-series-jul-2022/sample_submission.csv")
submission_lgbm['Predicted'] = np.argmax(lgbm_predictions_prob, axis = 1)
submission_lgbm.to_csv("submission_lgbm.csv",index = False)

CPU times: user 180 ms, sys: 3 ms, total: 183 ms
Wall time: 189 ms


In [27]:
%%time
submission_soft_vote = pd.read_csv("../input/tabular-playground-series-jul-2022/sample_submission.csv")
submission_soft_vote['Predicted'] = soft_vote_results
submission_soft_vote.to_csv("submission_softvote.csv",index = False)

CPU times: user 171 ms, sys: 4.97 ms, total: 176 ms
Wall time: 177 ms


In [28]:
%%time
# Placeholder, Describe Code...
submission_soft_vote.head(10)

CPU times: user 112 µs, sys: 3 µs, total: 115 µs
Wall time: 120 µs


,Id,Predicted
0,0,3
1,1,6
2,2,3
3,3,4
4,4,0
5,5,6
6,6,4
7,7,3
8,8,6
9,9,2
